In [7]:
# imports
import mysql.connector as sqltor
import os


In [8]:
# print a banner
print('**************************')
print('* Bank Management System *'.upper())
print('**************************')


**************************
* BANK MANAGEMENT SYSTEM *
**************************


In [9]:
CORRECT_USER_NAME = 'maharnab'
CORRECT_PASSWORD = 'maharnab'


In [10]:
# Login to the system
print('Enter credentials to log in to the system'.upper())
print('(Maximum 3 login attempts are allowed)\n')

userName = ''
password = ''

# Should fail after 3 login attempts
for i in range(3):
    userName = input('Enter the user name: ')
    password = input(f'Enter the password for {userName}: ')
    # validate entered username and password
    if userName != CORRECT_USER_NAME or password != CORRECT_PASSWORD:
        print('> ERROR: Wrong user name or password!')
        print('> Remaining', 3-i-1, 'attempts!')
    else:
        print('> Successfully logged in.')
        break
else:
    print('> [ERROR] Failed to login! EXIT!')
    ######################## UNCOMMENT ########################
    # quit()


ENTER CREDENTIALS TO LOG IN TO THE SYSTEM
(Maximum 3 login attempts are allowed)

> Successfully logged in.


In [11]:
# connect to mysql
print('> Initiailizing bank management system...')
print('> Logging into the database...')
dbCon = sqltor.connect(host='localhost', user=userName, passwd=password)
if dbCon.is_connected() == True:
    print('> Successfully logged into the database...')
else:
    print('> [ERROR] Failed logged into the database. EXIT')


> Initiailizing bank management system...
> Logging into the database...
> Successfully logged into the database...


In [12]:
dbCursor = dbCon.cursor()


In [13]:
#######################################################################
#                         DATABASE CREATION
#######################################################################

# create database if not already exists
databaseName = 'bank_db'
dbCursor.execute('CREATE DATABASE IF NOT EXISTS ' + databaseName)
dbCursor.execute('USE ' + databaseName)


In [14]:
#######################################################################
#                         TABLE CREATION
#######################################################################
# check accounts table
print('> Checking accounts table...')
accountTableName = 'bank_accs'
createTableQuery = f'''CREATE TABLE IF NOT EXISTS {accountTableName} (
        accno       CHAR(10)    NOT NULL PRIMARY KEY,
        acctype     CHAR(4)     NOT NULL,
        accname     VARCHAR(20),
        maincustid  INT NOT NULL,
        jtcustid    INT,
        createdate  DATE,
        updatedate  DATE,
        balance     DECIMAL
    )
'''
dbCursor.execute(createTableQuery)
dbCursor.execute('SELECT * FROM ' + accountTableName)
################## TBC????
# data = dbCursor.fetchall()
print(f'> Found details of {dbCursor.rowcount} accounts...')


> Checking accounts table...
> Found details of 0 accounts...


In [16]:
# check customer table
customerTableName = 'bank_custs'
print('> Checking customer table...')
customerTableName = 'bank_custs'
createTableQuery = f'''CREATE TABLE IF NOT EXISTS {customerTableName} (
        custid      INT UNSIGNED NOT NULL AUTO_INCREMENT PRIMARY KEY,
        custname    VARCHAR(30),
        mobile      CHAR(10),
        address     VARCHAR(50)
    )
'''
dbCursor.execute(createTableQuery)
dbCursor.execute('SELECT * FROM ' + customerTableName)
################## TBC????
# data = dbCursor.fetchall()
print(f'> Found details of {dbCursor.rowcount} customers...')

> Checking customer table...
> Found details of 0 customers...


In [24]:
#######################################################################
#                           DISPLAY MENU
#######################################################################

def displayMenu():
    print('+--------------------------------------------+')
    print('| BANK OPERATION MENU                        |')
    print('|                                            |')
    print('|   Available operations                     |')
    print('|                                            |')
    print('|   1. Open bank account                     |')
    print('|   2. Get account details                   |')
    print('|   3. Delete bank account                   |')
    print('|   4. Transact                              |')
    print('|   5. Get customer details                  |')
    print('|   6. Update customer details               |')
    print('|   7. Delete customer                       |')
    print('|   8. Clear screen and show menu            |')
    print('|   9. Quit                                  |')
    print('+--------------------------------------------+\n')    


In [25]:
displayMenu()


+--------------------------------------------+
| BANK OPERATION MENU                        |
|                                            |
|   Available operations                     |
|                                            |
|   1. Open bank account                     |
|   2. Get account details                   |
|   3. Delete bank account                   |
|   4. Transact                              |
|   5. Get customer details                  |
|   6. Update customer details               |
|   7. Delete customer                       |
|   8. Clear screen and show menu            |
|   9. Quit                                  |
+--------------------------------------------+



```python
        accno       CHAR(10)    NOT NULL PRIMARY KEY,
        acctype     CHAR(4)     NOT NULL,
        accname     VARCHAR(20),
        maincustid  INT NOT NULL,
        jtcustid    INT,
        createdate  DATE,
        updatedate  DATE,
        balance     DECIMAL
```

In [27]:
def getCustDetail():
    print('enter customer details'.upper())
    name = input('  Customer name? ')
    mobile = input('  Mobile number? ')
    address = input('  Address? ')
    
    return (name, mobile, address)

In [39]:
def createCustId(accType='SBAC'):
    # get number of specific type of accounts from db
    dbCon = sqltor.connect(host='localhost', user='maharnab', 
                           passwd='maharnab', database='bank_db')
    dbCursor = dbCon.cursor()
    dbCursor.execute(
        f"SELECT * FROM bank_accs WHERE acctype = '{accType}'")
    count = dbCursor.rowcount
    if count <= 0:
        count = 1
    
    return (accType[0] + str(count).zfill(9))
    

In [53]:
def addCustToDb(custId, name, mobile, address):
    dbCon = sqltor.connect(host='localhost', user='maharnab', 
                           passwd='maharnab', database='bank_db')
    dbCursor = dbCon.cursor()
    query = f'''
        INSERT INTO bank_custs(custname, mobile, address) VALUES(
            "{name}", "{mobile}", "{address}")
    '''
    print(query)
    dbCursor.execute(query)

In [55]:
addCustToDb("S000000001", "test_name", "9880561182", "Bangalore, 560024")



        INSERT INTO bank_custs(custname, mobile, address) VALUES(
            "test_name", "9880561182", "Bangalore, 560024")
    


In [ ]:
def openBankAcc():
    print('Creating a new bank account'.upper())
    
    # Choose account type
    accType = "SBAC"
    prefix = 'S'
    print('\n  step 1 - account type selection'.capitalize())
    print('  1. Savings\n  2. Current\n  3. Fixed deposit\n)')
    typeInp = input('Choose account type (1/2/3): ')
    if typeInp == 1:
        accType = "SBAC"
        prefix = 'S'
    elif typeInp == 2:
        accType = "CUAC"
        prefix = 'C'
    elif typeInp == 3:
        accType = "FDAC"
        prefix = 'F'
    else:
        print('> [ERROR] Failed! Wrong account type ({typeInp})!')
        return False
    
    # Main customer details
    print('\n  step 2 - main customer selection'.capitalize())
    existingCust = input('  Existing customer (Yes/No)? ')
    if existingCust[0].lower() == 'y':
        mainCustId = input('  Customer id? ')
    else:
        (name, mobile, address) = getCustDetail()
        custId = createCustId(accType)
    
    return True

In [ ]:
choice = 0
while choice != 9:
    displayMenu()
    choice = int(input('>>> Enter your choice [1-9]:'))
    if choice == 1:
        openBankAcc()

In [19]:
os.system('cls' if os.name == 'nt' else 'clear')


0

In [8]:
def login():
    print('Log in to the system\n')
    # Should fail after 3 login attempts
    for i in range(3):
        userName = input('Enter the user name: ')
        password = input('Enter the password: ')
        # validate entered username and password
        if userName != 'maharnab' or password != 'maharnab':
            print('ERROR: Wrong user name or password!')
            print('Remaining', 3-i-1, 'attempts!')
        else:
            return True
    else:
        print('[ERROR] Failed to login!')
        return False
  

In [21]:
customerTableName = 'bank_custs'
createTableQuery = f'''CREATE TABLE IF NOT EXISTS {customerTableName} (
        custid      INT UNSIGNED NOT NULL AUTO_INCREMENT,
        custname    VARCHAR(30),
        mobile      CHAR(10),
        address     VARCHAR(50),
        accno       CHAR(10),
        PRIMARY KEY (custid, accno),
        FOREIGN KEY (accno) REFERENCES bank_accs(accno)
    )
'''
print(createTableQuery)

CREATE TABLE IF NOT EXISTS bank_custs (
        custid      INT UNSIGNED NOT NULL AUTO_INCREMENT,
        custname    VARCHAR(30),
        mobile      CHAR(10),
        address     VARCHAR(50),
        accno       CHAR(10),
        PRIMARY KEY (custid, accno),
        FOREIGN KEY (accno) REFERENCES bank_accs(accno)
    )



In [20]:
n = 123
string_val = 'FD' + str(n).zfill(8)
print(string_val)


FD00000123


In [9]:
login()


Log in to the system

ERROR: Wrong user name or password!
Remaining 2 attempts!
ERROR: Wrong user name or password!
Remaining 1 attempts!
ERROR: Wrong user name or password!
Remaining 0 attempts!
[ERROR] Failed to login!


False

In [1]:
import mysql.connector as sqltor

myCon = sqltor.connect(host='localhost', user='maharnab', passwd='maharnab')

if myCon.is_connected() == True:
    print("Success")

Success
